The aim of this experiment is simply to test that MS-RWM outperforms MS-RWM on the G and K example.

In [1]:
import numpy as np
from numpy.random import rand, randn
from numpy import ones, exp, log, diag, vstack, pi, array, r_, isfinite, logspace, zeros, eye, quantile
from numpy.linalg import norm, solve
from numpy.random import default_rng, choice, randint
from scipy.optimize import fsolve
from scipy.stats import multivariate_normal as MVN
from scipy.special import ndtri, ndtr
from scipy.stats import uniform as udist
from scipy.stats import norm as ndist
from scipy.linalg import block_diag

import time
from math import prod
from warnings import catch_warnings, filterwarnings, resetwarnings


import matplotlib.pyplot as plt
from matplotlib import rc
from ipywidgets.widgets import interact


from RWM import RWM
from Manifolds.Manifold import Manifold
from Manifolds.GKManifoldNew import GKManifold, data_generator, find_point_on_manifold, find_point_on_manifold_from_θ
from markov_snippets import MSAdaptiveTolerancesAdaptiveδ, MSAdaptiveTolerancesAdaptiveδSwitchIntegrator
# from tangential_hug_functions import HugTangentialMultivariate
# from utils import ESS_univariate, prep_contour
from copy import deepcopy

# G and K settings

In [2]:
def generate_setting(m, B, δ, N, ϵ0=20, thinning=10, thug=True, verbose=True, initialization='init_RWMϵ0', burnin=100, δ0=0.01, kernel_type='uniform', ϵmin=1e-4, quantile_value=0.01, maxiter=50, ϵprop_switch=0.05, pmoved_switch=0.15, switch_strategy='ap', δmin=1e-4, δmax=100.0, ap_target=0.3, use_autograd=False, init_manifold_prior=True):
    """Generates an object from which one can grab the settings. This allows one to run multiple scenarios."""
    θ0        = array([3.0, 1.0, 2.0, 0.5])      # True parameter value on U(0, 10) scale.
    d         = 4 + m                            # Dimensionality of ξ=(θ, z)
    ystar     = data_generator(θ0, m, seed=1234) # Observed data
    q         = MVN(zeros(d), eye(d))            # Proposal distribution for THUG
    ξ0        = find_point_on_manifold_from_θ(ystar=ystar, θfixed=ndtri(θ0/10), ϵ=1e-5, maxiter=5000, tol=1e-15, kernel_type=kernel_type)
    resetwarnings()
    manifold  = GKManifold(ystar, kernel_type=kernel_type, use_autograd=use_autograd)
    if δ0 is None:
        δ0 = δ
    return {
        'θ0': θ0,
        'm' : m,
        'd' : d,
        'ystar': ystar,
        'q': q,
        'ξ0': ξ0,
        'B': B,
        'δ': δ,
        'N': N,
        'manifold': manifold,
        'thinning': thinning,
        'thug': thug,
        'verbose': verbose,
        'initialization': initialization,
        'burnin': burnin,
        'δ0': δ0,
        'kernel_type': kernel_type,
        'ϵmin': ϵmin,
        'quantile_value': quantile_value,
        'maxiter': maxiter,
        'ϵprop_switch': ϵprop_switch,
        'pmoved_switch': pmoved_switch,
        'switch_strategy': switch_strategy,
        'δmin': δmin,
        'δmax': δmax,
        'ap_target': ap_target,
        'ϵ0': ϵ0,
        'use_autograd': use_autograd,
        'init_manifold_prior': init_manifold_prior
    }

# Markov Snippets RWM with adaptive step size

In [3]:
SETTINGS = generate_setting(
    m=50, 
    B=20, 
    δ=0.01, 
    N=5000, 
    thinning=10, 
    δ0=0.1, 
    kernel_type='uniform', 
    thug=False, 
    verbose=True, 
    initialization='init_prior', 
    burnin=100, 
    maxiter=200, 
    ϵmin=1e-8, 
    quantile_value=0.01,
    δmin=1e-3, 
    δmax=10.0, 
    ap_target=0.2
)

In [ ]:
MS_RWM = MSAdaptiveTolerancesAdaptiveδ(SETTINGS)
_ = MS_RWM.sample()

Integrator: RWM.
Initializing particles from prior.
Iteration:  1
	Trajectories constructed.
	Epsilon: 29.524731
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.976
	Step-size adapted to: 0.01474325
Iteration:  2
	Trajectories constructed.
	Epsilon: 17.600537
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 1.000
	Step-size adapted to: 0.02199434
Iteration:  3
	Trajectories constructed.
	Epsilon: 15.669196
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.905
	Step-size adapted to: 0.03128958
Iteration:  4
	Trajectories constructed.
	Epsilon: 13.740547
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.959
	Step-size adapted to: 0.04574054
Iteration:  5
	Trajectories constructed.
	Epsilon: 12.252569
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.744
	Step-size adapte

	Epsilon: 6.577686
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.187
	Step-size adapted to: 0.05235174
Iteration:  44
	Trajectories constructed.
	Epsilon: 6.505337
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.169
	Step-size adapted to: 0.05154139
Iteration:  45
	Trajectories constructed.
	Epsilon: 6.441169
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.196
	Step-size adapted to: 0.05144355
Iteration:  46
	Trajectories constructed.
	Epsilon: 6.380588
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.145
	Step-size adapted to: 0.05003812
Iteration:  47
	Trajectories constructed.
	Epsilon: 6.303705
	Weights computed and normalized.
	Particles Resampled.
	Velocities refreshed.
	Prop Moved: 0.189
	Step-size adapted to: 0.04977362
Iteration:  48
	Trajectories constructed.
	Epsilon: 6.245025
	Weights compute